In [3]:
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import gc
from datetime import datetime 
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

In [4]:
pip install xlrd

  Using cached xlrd-2.0.1-py2.py3-none-any.whl.metadata (3.4 kB)
Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import necessary libraries
import pandas as pd

# Correct file path (use the uploaded file location)
file_path = '/mnt/data/default of credit card clients.xls'

# Load the Excel file
try:
    data = pd.read_excel(file_path, header=1, engine='xlrd')  # Use header=1 to skip the first row
    print("Data loaded successfully!")
    print(data.head())
except FileNotFoundError:
    print(f"File not found: {file_path}. Please check the file path.")
except Exception as e:
    print(f"An error occurred: {e}")

# Optional: Display additional information about the dataset
try:
    print("\nDataset Information:")
    print(data.info())
    print("\nDataset Shape:", data.shape)
except Exception as e:
    print(f"Unable to process the dataset: {e}")

File not found: /mnt/data/default of credit card clients.xls. Please check the file path.

Dataset Information:
Unable to process the dataset: name 'data' is not defined


In [ ]:
print("Default Credit Card Clients data -  rows:",data.shape[0]," columns:", data.shape[1])

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
total = data.isnull().sum().sort_values(ascending=False)
percent = data.isnull().sum()/data.shape[0]*100
pd.concat([total, percent], axis=1, keys=['Total', 'Percent']).transpose()
#pd.concat(): This function is used to concatenate two or more pandas objects (like DataFrames or Series) along a particular axis.

In [1]:
temp = data["default payment next month"].value_counts(normalize=True)
df = pd.DataFrame({'default payment next month': temp.index,'values': temp.values})

plt.figure(dpi = 140)
plt.title('Checking data imbalance \n (Default = 0, Not Default = 1)')
plt.bar(df['default payment next month'], df['values'])
plt.show()

NameError: name 'data' is not defined

In [3]:
plt.figure(figsize = (14,6))
plt.title("Amount of credit limit - Density Plot")
sns.set_color_codes("pastel")
sns.distplot(data['LIMIT_BAL'],kde=True,bins=200,color="blue")
plt.show()

NameError: name 'plt' is not defined

In [5]:
data['LIMIT_BAL'].value_counts().shape

NameError: name 'data' is not defined

In [ ]:
data['LIMIT_BAL'].value_counts()

In [ ]:
grouped_data = data.groupby('default payment next month')["LIMIT_BAL"]

class_0 = grouped_data.get_group(0)
class_1 = grouped_data.get_group(1)

In [ ]:
class_1

In [ ]:
class_0

In [ ]:
plt.figure(figsize = (14,6))
plt.title('Default amount of credit limit  - grouped by Payment Next Month (Density Plot)')
sns.set_color_codes("pastel")
sns.distplot(class_1,kde=True,bins=200, color="red")
sns.distplot(class_0,kde=True,bins=200, color="green")
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12,6))
s = sns.boxplot(ax = ax1, x="SEX", y="LIMIT_BAL", hue="SEX",data=data, palette="PRGn",showfliers=True)
s = sns.boxplot(ax = ax2, x="SEX", y="LIMIT_BAL", hue="SEX",data=data, palette="PRGn",showfliers=False)
plt.show()
#showfliers=True: Shows outliers in the boxplot (points that fall outside the whiskers of the boxplot)
#ax1, ax2: The two subplot axes where the boxplots will be drawn. ax1 will be the first subplot (left), and ax2 will be the second subplot (right).

In [ ]:
data.info()

In [ ]:
var = ['PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5']

plt.figure(figsize = (8,8))
plt.title('Amount of previous payment (Apr-Sept) \ncorrelation plot')
corr = data[var].corr()
sns.heatmap(corr,xticklabels=corr.columns,yticklabels=corr.columns,linewidths=.1,
            vmin=-1, vmax=1)
plt.show()

In [ ]:
var = ['PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6']

plt.figure(figsize = (8,8))
plt.title('Repayment status (Apr-Sept) \ncorrelation plot')
corr = data[var].corr()
sns.heatmap(corr,xticklabels=corr.columns,yticklabels=corr.columns,linewidths=.1,vmin=-1, vmax=1)
plt.show()

In [ ]:
def boxplot_variation(feature1, feature2, feature3, width=16):
    fig, ax1 = plt.subplots(ncols=1, figsize=(width,6))
    s = sns.boxplot(ax = ax1, x=feature1, y=feature2, hue=feature3,
                data=data, palette="PRGn",showfliers=False)
    s.set_xticklabels(s.get_xticklabels(),rotation=90)
    plt.show()

In [ ]:
boxplot_variation('MARRIAGE','AGE', 'SEX',8)

In [ ]:
boxplot_variation('EDUCATION','AGE', 'MARRIAGE',12)

In [ ]:
boxplot_variation('AGE','LIMIT_BAL', 'SEX',16)

In [ ]:
boxplot_variation('MARRIAGE','LIMIT_BAL', 'EDUCATION',12)

In [ ]:
target = 'default.payment.next.month'
predictors = [  'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 
                'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 
                'BILL_AMT1','BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
                'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']

In [ ]:
train_df, val_df = train_test_split(data, test_size=0.25, random_state=42, shuffle=True )

In [ ]:
train_df_copy = train_df.copy()
val_df_copy = val_df.copy()

In [ ]:
classifier = RandomForestClassifier(class_weight='balanced',
                                    n_jobs = -1,
                                   random_state = 42,
                                   criterion='gini',
                                   n_estimators = 350,
                                   verbose = False)
#No output or minimal output is shown.

In [ ]:
predictors = train_df.drop(columns = ['default payment next month']).columns.values
target = 'default payment next month'

In [ ]:
classifier.fit(train_df[predictors].values, train_df[target].values)
#using .values extracts the underlying NumPy array from the DataFrame or Series

In [ ]:
preds = classifier.predict(val_df[predictors])

In [ ]:
tmp = pd.DataFrame({'Feature': predictors, 'Feature importance': classifier.feature_importances_})
tmp = tmp.sort_values(by='Feature importance',ascending=False)
plt.figure(figsize = (7,4))
plt.title('Features importance',fontsize=14)
s = sns.barplot(x='Feature',y='Feature importance',data=tmp)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.show()   
#clf.feature_importances_: This is an attribute from the trained model clf(likely a tree-based model like RandomForest or GradientBoosting) that provides the importance scores for each feature.

In [ ]:
cm = pd.crosstab(val_df[target].values, preds, rownames=['Actual'], colnames=['Predicted'])
fig, (ax1) = plt.subplots(ncols=1, figsize=(5,5))
sns.heatmap(cm, 
            xticklabels=['Not Default','Default'],
            yticklabels=['Not Default','Default'],
            annot=True,ax=ax1,
            linewidths=.2,linecolor="Darkblue", cmap="Reds")
plt.title('Confusion Matrix', fontsize=14)
plt.show()
#cmap="Blues": Sets the color map of the heatmap to shades of blue

In [ ]:
roc_auc_score(val_df[target].values, preds)